In [48]:
import pandas as pd
import gzip
import wget
import gensim
import nltk
from nltk.collocations import *

In [2]:
def parse(path):
    g = gzip.open(path, 'rb')
    true = True
    false = False
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

df = getDF(r'C:\\Users\\Лера\\Musical_Instruments.json.gz')

Объединим два датафрейма (с отзывами и метаданными) по значению asin

In [3]:
df_reviews = df[['reviewText', 'asin']]

In [4]:
df_reviews.head(5)

,reviewText,asin
0,Crocheting for Dummies by Karen Manthey & Susa...,0470536454
1,Very helpful...,0470536454
2,EASY TO UNDERSTAND AND A PROMPT SERVICE TOO,0470536454
3,My girlfriend use quite often,0470536454
4,Arrived as described. Very happy.,0470536454


In [5]:
df2 = getDF(r'C:\\Users\\Лера\\meta_Musical_Instruments.json.gz')

In [6]:
df2.head(3)

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
0,"[Musical Instruments, Drums & Percussion, Hand...",,[Cricket Rubbing the spine with the wooden sti...,,Wooden Percussion 2 Piece Set of 3 Inch Cricke...,"[B00NP8GYVS, B00NP80XMO, B00NP8M098]",,WADSUWAN SHOP,"[Wood percussion, Owl whistle*, Includes woode...","[>#141,729 in Musical Instruments (See Top 100...",[],Musical Instruments,,"December 2, 2013",,0000989983,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
1,"[Musical Instruments, Drums & Percussion, Hand...",,[Frog - Rubbing its spine with the wooden stic...,,"Wooden Percussion 3 Piece Set Frog, Cricket an...","[B00NP8GYVS, B00NP80XMO, B01MY48HK5, B00AZZ1AJ...",,WADSUWAN SHOP,"[Wood percussion, Small 3 inches, Creates orig...","[>#1,622 in Musical Instruments (See Top 100 i...",[],Musical Instruments,,"December 2, 2013",$0.91,0000098906,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
2,"[Musical Instruments, Instrument Accessories, ...",,[Vivaldi's famous set of four violin concertos...,,Hal Leonard Vivaldi Four Seasons for Piano (Or...,[],,Hal Leonard,"[., ., .]","[>#330,653 in Musical Instruments (See Top 100...",[],Musical Instruments,,"May 10, 2011",$62.93,0041291905,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN


In [7]:
df_meta = df2[['category', 'title', 'asin']]

In [8]:
df_meta.head(5)

,category,title,asin
0,"[Musical Instruments, Drums & Percussion, Hand...",Wooden Percussion 2 Piece Set of 3 Inch Cricke...,0000989983
1,"[Musical Instruments, Drums & Percussion, Hand...","Wooden Percussion 3 Piece Set Frog, Cricket an...",0000098906
2,"[Musical Instruments, Instrument Accessories, ...",Hal Leonard Vivaldi Four Seasons for Piano (Or...,0041291905
3,[],The Turn of the Screw (vocal score),0060015500
4,[],Suite for Organ (including the Trumpet Volunta...,0193757710


In [9]:
res = df_reviews.merge(df_meta, on=["asin"])

In [10]:
res.head(5)

,reviewText,asin,category,title
0,Crocheting for Dummies by Karen Manthey & Susa...,0470536454,"[Musical Instruments, Instrument Accessories, ...",Wiley Publishers Crocheting For Dummies Revised
1,Very helpful...,0470536454,"[Musical Instruments, Instrument Accessories, ...",Wiley Publishers Crocheting For Dummies Revised
2,EASY TO UNDERSTAND AND A PROMPT SERVICE TOO,0470536454,"[Musical Instruments, Instrument Accessories, ...",Wiley Publishers Crocheting For Dummies Revised
3,My girlfriend use quite often,0470536454,"[Musical Instruments, Instrument Accessories, ...",Wiley Publishers Crocheting For Dummies Revised
4,Arrived as described. Very happy.,0470536454,"[Musical Instruments, Instrument Accessories, ...",Wiley Publishers Crocheting For Dummies Revised


In [11]:
res['category'][1731694]

['Musical Instruments', 'Guitars', 'Electric Guitars', 'Solid Body']

Оставляем в датафрейме только информацию о гитарах

In [12]:
g = 'Guitars'
guitars = res[res["category"].apply(lambda x: g in x)]

In [13]:
guitars.head(5)

,reviewText,asin,category,title
1024,I got this on sale for $49.99 and fro that muc...,0739079883,"[Musical Instruments, Guitars, Acoustic Guitar...","Alfred's Kid's Guitar Course, Complete Starter..."
1025,This is a good starter package. It came with a...,0739079883,"[Musical Instruments, Guitars, Acoustic Guitar...","Alfred's Kid's Guitar Course, Complete Starter..."
1026,I ordered the Guitar for my Granddaughter for ...,0739079883,"[Musical Instruments, Guitars, Acoustic Guitar...","Alfred's Kid's Guitar Course, Complete Starter..."
1027,I bought this pack as a gift for a friend's ch...,0739079883,"[Musical Instruments, Guitars, Acoustic Guitar...","Alfred's Kid's Guitar Course, Complete Starter..."
1028,Got this for my 10yr old daughter. She enjoys...,0739079883,"[Musical Instruments, Guitars, Acoustic Guitar...","Alfred's Kid's Guitar Course, Complete Starter..."


Для выделения ne из текстов отзывов можно попробовать следующие способы:

1. Составить мини-словарь сущностей/дескрипторов, расширить с помощью слов из модели (например, word2vec). Мне кажется, что это нормальный способ, так как можно вручную задать употребления типа this pack, that item итд, которые не называют инструмент прямо, но обозначают его в отзыве. Из минусов - вряд ли здесь можно учесть все случаи, когда пользователь называет конкретную модель (кроме самых популярных) - этот способ буду реализовывать ниже. Можно дополнить словарь синонимами из wordnet.
2. Можно попробовать выделить морфологические и орфографические шаблоны со списком основных существительных: например, берем слова guitar, device, set итд, предполагаем, что частыми морфологическими шаблонами будут ADJ + NOUN и проверяем биграммы из отзывов на соответствие этим шаблонам. Орфографические шаблоны: можно брать тот же список основных существительных и выделять би и триграммы, в которых другое/другие слова пишутся с большой буквы, с большой вероятностью, это будет название модели. Из плюсов - мы сможем выделить названия моделей товаров, которые вряд ли найдутся в предобученной модели или в синонимах wordnet. 
3. Использовать столбцы category и title из метаданных, считая все коллокации там упоминаниями товара, которые будем искать в отзывах. Плюс - готовые данные, которые с большой вероятностью встретятся в отзывах.

В первых двух способах основные данные - тексты отзывов, метаданные были вспомогательными для очистки датасета. В третьем способе работаем с метаданными.

Попробуем найти близкие слова в предобученной модели

In [14]:
import gensim.downloader

In [15]:
glove_vectors = gensim.downloader.load('glove-twitter-25')

In [16]:
glove_vectors.most_similar('guitar')

[('theme', 0.9055464863777161),
 ('cover', 0.902897834777832),
 ('bass', 0.9020538330078125),
 ('drum', 0.8947978615760803),
 ('drums', 0.8919881582260132),
 ('iron', 0.8911890983581543),
 ('shuffle', 0.8809096813201904),
 ('keys', 0.8803252577781677),
 ('mic', 0.8774563670158386),
 ('magic', 0.8769288659095764)]

In [17]:
glove_vectors.most_similar('item')

[('model', 0.8291915059089661),
 ('xl', 0.7993069291114807),
 ('limited', 0.7985807061195374),
 ('custom', 0.7905901074409485),
 ('dp', 0.7867296934127808),
 ('print', 0.7839182615280151),
 ('bib', 0.7831104397773743),
 ('pen', 0.7819660305976868),
 ('kit', 0.7776104807853699),
 ('slr', 0.7739841938018799)]

Можно попробовать вытащить синонимы из wordnet и дополнить ими словарь

In [18]:
from nltk.corpus import wordnet
wordnet.synset('guitar.n.01').lemma_names()

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.o

['guitar']

In [19]:
wordnet.synset('item.n.01').lemma_names()

['item', 'point']

Составляем словарь

In [21]:
vocab = ['guitar', 'instrument', 'set', 'model', 'pack', 'item']

In [22]:
reviews = guitars['reviewText'].tolist()

In [52]:
str_reviews = []
for r in reviews:
    r = str(r)
    str_reviews.append(r)

In [25]:
list_of_reviews = []
for i in reviews:
    i = str(i)
    i = i.split(' ')
    list_of_reviews.extend(i)

In [28]:
reduced_list_of_reviews = list_of_reviews[0:1000]

Находим коллокации

In [37]:
left_bigrams = []
right_bigrams = []
for idx, item in enumerate(list_of_reviews):
    left_collocation = []
    right_collocation = []
    for word in vocab:
        if item == word:
            if list_of_reviews[idx - 1] == 'this' or list_of_reviews[idx - 1] == 'that' or list_of_reviews[idx - 1] == 'the' or list_of_reviews[idx - 1] == 'The':
                left_collocation.append(list_of_reviews[idx - 2])
                left_collocation.append(list_of_reviews[idx - 1])
                left_collocation.append(list_of_reviews[idx])
                right_collocation.append(list_of_reviews[idx - 1])
                right_collocation.append(list_of_reviews[idx])
                right_collocation.append(list_of_reviews[idx + 1])
            else:
                left_collocation.append(list_of_reviews[idx - 1])
                left_collocation.append(list_of_reviews[idx])
                right_collocation.append(list_of_reviews[idx])
                right_collocation.append(list_of_reviews[idx + 1])
    left_bigrams.append(left_collocation)
    right_bigrams.append(right_collocation)

In [39]:
left_bigrams_clean = [x for x in left_bigrams if x != []] #убираем пустые списки
right_bigrams_clean = [x for x in right_bigrams if x != []]

Делим все коллокации на биграммы и триграммы

In [63]:
true_bigrams = []
true_trigrams = []

for el in left_bigrams_clean:
    if len(el) == 2:
        true_bigrams.append(el)
    elif len(el) == 3:
        true_trigrams.append(el)

for el in right_bigrams_clean:
    if len(el) == 2:
        true_bigrams.append(el)
    elif len(el) == 3:
        true_trigrams.append(el)

In [66]:
true_bigrams[2:7]

[['fancier', 'guitar'],
 ['starter', 'set'],
 ['Good', 'guitar'],
 ['This', 'guitar'],
 ['level', 'guitar']]

In [67]:
true_trigrams[2:7]

[['bought', 'this', 'pack'],
 ['on', 'the', 'instrument'],
 ['review.', 'The', 'guitar'],
 ['tune', 'the', 'guitar'],
 ['bought', 'this', 'guitar']]

Ранжируем биграммы и триграммы

In [76]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()
finder2 = BigramCollocationFinder.from_documents(true_bigrams)
finder2.apply_freq_filter(10)

In [86]:
finder2.score_ngrams(bigram_measures.likelihood_ratio)[:10]

[(('set', 'up'), 13893.085021373972),
 (('guitar', 'set'), 13870.753723322023),
 (('guitar', 'for'), 9815.258566698134),
 (('for', 'guitar'), 8101.241089798707),
 (('This', 'guitar'), 7851.691422961571),
 (('guitar', 'a'), 6902.001492511218),
 (('a', 'guitar'), 6497.1633837403515),
 (('guitar', 'This'), 5848.929620348293),
 (('set', 'of'), 5662.405683755949),
 (('is', 'guitar'), 5254.070152996523)]

In [87]:
finder2.score_ngrams(bigram_measures.pmi)[:10]

[(('battery', 'pack'), 8.171783988558568),
 (('combo', 'pack'), 7.01424271157209),
 (('floor', 'model'), 6.1860266757339275),
 (('model', 'number'), 6.17537943153442),
 (('newer', 'model'), 6.17537943153442),
 (('older', 'model'), 6.03716328981945),
 (('sunburst', 'model'), 5.968928554066995),
 (('base', 'model'), 5.9148518813112005),
 (('signature', 'model'), 5.490881257262352),
 (('musical', 'instrument'), 5.3425691077365975)]

In [88]:
finder2.score_ngrams(bigram_measures.dice)[:10]

[(('set', 'up'), 0.2767338571916638),
 (('set', 'of'), 0.17124339075558587),
 (('an', 'instrument'), 0.13045779306694893),
 (('guitar', 'for'), 0.1285451553576614),
 (('a', 'guitar'), 0.10551093879811686),
 (('This', 'guitar'), 0.09577246623987608),
 (('a', 'set'), 0.07806037138079733),
 (('guitar', 'is'), 0.07724509119904316),
 (('instrument', 'that'), 0.06703470031545741),
 (('guitar', 'and'), 0.0645327783558793)]

In [78]:
finder3 = TrigramCollocationFinder.from_documents(true_trigrams)
finder3.apply_freq_filter(10)

In [89]:
finder3.score_ngrams(trigram_measures.likelihood_ratio)[:10]

[(('this', 'guitar', 'the'), 180218.32152956477),
 (('the', 'guitar', 'this'), 163885.9133423194),
 (('this', 'guitar', 'is'), 155419.73601409938),
 (('the', 'the', 'guitar'), 153200.74328485475),
 (('guitar', 'The', 'guitar'), 151024.30932892964),
 (('the', 'guitar', 'the'), 150318.68355224),
 (('this', 'guitar', 'for'), 145839.49066218318),
 (('bought', 'this', 'guitar'), 145724.68572749523),
 (('of', 'this', 'guitar'), 145611.15112874412),
 (('is', 'this', 'guitar'), 141979.92789457444)]

In [90]:
finder3.score_ngrams(trigram_measures.pmi)[:10]

[(('The', 'set', 'neck'), 9.694537045239795),
 (('The', 'set', 'up'), 8.961052019808548),
 (('the', 'set', 'up.'), 8.950876993564979),
 (('the', 'set', 'up'), 8.46033009473593),
 (('the', 'set', 'neck'), 8.07833790274724),
 (('return', 'the', 'item'), 7.919603704166377),
 (('this', 'set', 'up.'), 7.824472212661188),
 (('returned', 'the', 'item'), 7.206536377697539),
 (('The', 'item', 'arrived'), 7.005800166170069),
 (('in', 'the', 'pack'), 6.915039685093305)]

In [91]:
finder3.score_ngrams(trigram_measures.student_t)[:10]

[(('this', 'guitar', 'is'), 50.23507495494184),
 (('this', 'guitar', 'for'), 48.641565163970526),
 (('of', 'the', 'guitar'), 42.73335609103468),
 (('bought', 'this', 'guitar'), 40.317948591692904),
 (('the', 'guitar', 'is'), 38.79273773401875),
 (('The', 'guitar', 'is'), 35.318880516112856),
 (('', 'The', 'guitar'), 33.65597294150676),
 (('the', 'guitar', 'and'), 30.22246756091399),
 (('recommend', 'this', 'guitar'), 29.695189984111703),
 (('this', 'guitar', 'and'), 29.64262734186651)]

Для биграмм наиболее адекватными выглядят результаты с метрикой pmi, для триграмм - в принципе во всех трех есть разумные вещи типа 'recommend', 'this', 'guitar' или 'returned', 'the', 'item'. Вообще триграммы кажутся более подходящими для отчета, так как в них помещается не только название товара, но и примерное отношение пользователей